# Importing packages

In [1]:
using JuMP
using SCIP
using Formatting
using GLPKMathProgInterface

# Defining experiment parameters

In [2]:
instances_dir = "Problema2"
time_limit = 60 * 60; # seconds

# Running Experiment

In [16]:
# instance_files = readdir(instances_dir)
instance_files = ["smt_50_250.dat"]

for instance_file in instance_files

    print("Running $instance_file\n")
    
    instance_file_path = joinpath.(instances_dir, instance_file)

    f = open(instance_file_path, "r")         

    lines = readlines(f)

    close(f)

    n, m, s, t = [parse(Int, num_char) for num_char in split(lines[1])]

    V = Array(1:n)
    V_st = copy(V)
    deleteat!(V_st, V_st .== s)
    deleteat!(V_st, V_st .== t)

    D = zeros(Int64, (n, n))
    G = zeros(Int64, (n, n))
    P = zeros(Int64, (n, n, n))

    for i in 2:m+1
        u, v, d = [parse(Int, num_char) for num_char in split(lines[i])]
        D[u, v] = d
        D[v, u] = d
        G[u, v] = 1
        G[v, u] = 1
    end

    for i in 1:n
        for j in 1:n
            for k in 1:n
                P[i,j,k] = abs(D[i,j] - D[j,k])
            end
        end
    end

    M = maximum(P);

    # Defining Model

    model = Model(SCIP.Optimizer)

    set_time_limit_sec(model, time_limit);

    @variable(model, C[1:n, 1:n], Bin)
    @variable(model, Q[1:n, 1:n, 1:n], Bin)
    @variable(model, p >= 0, Int)

    @objective(model, Min, p)

    # Restrições para gerar um caminho C subgrafo de G
    @constraint(model, sum(C[j,s] for j = 1:n) == 0)
    @constraint(model, sum(C[s,j] for j = 1:n) == 1)
    @constraint(model, sum(C[j,t] for j = 1:n) == 1)
    @constraint(model, sum(C[t,j] for j = 1:n) == 0)
    @constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) <= 1)
    @constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) - sum(C[i,j] for j = 1:n) == 0)
    @constraint(model, [i = 1:n, j=1:n], C[i,j] - G[i,j] <= 0)

    # Restrições para que  Q_{ijk} = C_{ij} ^ C_{jk}
    @constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] <= (C[i,j] + C[j,k])/2)
    @constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] >= C[i,j] + C[j,k] - 1)

    # Restrições para que p seja o passo máximo do caminho selecionado
    @constraint(model, [i = 1:n, j=1:n, k=1:n], p >= P[i,j,k] + (-M * (1 - Q[i,j,k])));

    # Optimizing

    x = @timed optimize!(model)


    optimization_time = x[2]
    proved_optimality = termination_status(model) == MOI.OPTIMAL
    timeout = termination_status(model) == MOI.TIME_LIMIT
    
    print("\n<--------------------------------------------------------------------->\n\n")
    print("Results for $instance_file:\n")
    print("\toptimization_time: $optimization_time\n")
    print("\tobjective_value: ")
    print(objective_value(model))
    print("\n")
    print("\tproved_optimality: $proved_optimality\n")
    print("\ttimeout: $timeout\n")
    print("\n<--------------------------------------------------------------------->\n")

end

Running smt_50_250.dat
presolving:
(round 1, fast)       122138 del vars, 213152 del conss, 0 add conss, 122090 chg bounds, 44158 chg sides, 44158 chg coeffs, 0 upgd conss, 0 impls, 50 clqs
(round 2, fast)       122138 del vars, 363326 del conss, 0 add conss, 122090 chg bounds, 44158 chg sides, 44158 chg coeffs, 0 upgd conss, 0 impls, 50 clqs
   (2.0s) running MILP presolver
   (2.0s) MILP presolver found nothing
(round 3, exhaustive) 122138 del vars, 363326 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coeffs, 9346 upgd conss, 0 impls, 50 clqs
(round 4, exhaustive) 122163 del vars, 363326 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coeffs, 9346 upgd conss, 4416 impls, 77185 clqs
(round 5, fast)       122163 del vars, 363351 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coeffs, 9346 upgd conss, 4416 impls, 77185 clqs
(round 6, exhaustive) 122163 del vars, 363376 del conss, 0 add conss, 122091 chg bounds, 44158 ch

Results for smt_50_250.dat:
	optimization_time: 30.420939952
	objective_value: 501.0
	proved_optimality: true
	timeout: false

<--------------------------------------------------------------------->
